Step 1 : Drop the date column

Step 2 : Seperate the good data points from the bad data points.

Step 3 : Replace 'Good' and 'Bad' with 1 and 2 (The computer works with numbers, not strings). 

Step 4 : Dropping null values (No null values in our data set, therefore none dropped).

Step 5 : Seperating the data into two dataframes. One with the good labels, the other with the bad labels (1 and 2 respectively).

Step 6 : Drop the quality column. It is the label column, we only need the feature columns to build a model.

Step 7 : Convert the two dataframes into vecotrs (Vectors go into models, not dataframes).

Step 8 : Divide the data into training data and testing data.

Step 9 : Create the model, and train it with the training data.

Step 10 : Check out the results of the model


Note : We only use the good feature vector to train the model


In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
# changing working directory

os.chdir('C:\\Anomaly detection IR')
os.getcwd()

'C:\\Anomaly detection IR'

In [3]:
# importing the data 

df = pd.read_csv('anomaly_data.txt', sep = ',')
df.head()

,Date,Power,Detector,Quality
0,1-Jan-16,96,8,Good
1,2-Jan-16,96,10,Good
2,3-Jan-16,91,8,Good
3,4-Jan-16,97,9,Good
4,5-Jan-16,91,11,Good


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      1461 non-null   object
 1   Power     1461 non-null   int64 
 2   Detector  1461 non-null   int64 
 3   Quality   1461 non-null   object
dtypes: int64(2), object(2)
memory usage: 45.8+ KB


In [5]:
df.groupby('Quality')['Quality'].count()

Quality
Bad      407
Good    1054
Name: Quality, dtype: int64

In [6]:
# dropping date column because it's useless for us

df.drop(columns = 'Date', inplace = True)

In [7]:
df.head()


,Power,Detector,Quality
0,96,8,Good
1,96,10,Good
2,91,8,Good
3,97,9,Good
4,91,11,Good


In [8]:
df['Quality'].nunique()

2

In [9]:
# making a numeric column from quality column

df['Quality_num'] = np.where(df['Quality']=='Good', 1,2 )
df

,Power,Detector,Quality,Quality_num
0,96,8,Good,1
1,96,10,Good,1
2,91,8,Good,1
3,97,9,Good,1
4,91,11,Good,1
...,...,...,...,...
1456,93,8,Good,1
1457,91,10,Good,1
1458,95,8,Good,1
1459,91,10,Good,1


In [10]:
df['Quality_num'].nunique()

2

In [11]:
df.drop(columns = ['Quality'], inplace = True)

In [12]:
df.head()

,Power,Detector,Quality_num
0,96,8,1
1,96,10,1
2,91,8,1
3,97,9,1
4,91,11,1


In [13]:
df.describe()

,Power,Detector,Quality_num
count,1461.000000,1461.000000,1461.000000
mean,95.856947,9.980835,1.278576
std,8.435862,2.174803,0.448452
min,61.000000,5.000000,1.000000
25%,92.000000,8.000000,1.000000
50%,95.000000,10.000000,1.000000
75%,98.000000,11.000000,2.000000
max,148.000000,17.000000,2.000000


In [14]:
# reference for non anomaly data with good values
good_mask = df['Quality_num'] == 1

In [15]:
# reference for non anomaly data with bad values
bad_mask = df['Quality_num'] == 2

In [16]:
df.drop(columns = 'Quality_num', inplace = True)

In [17]:
# df with only good non anomaly values
df_good = df[good_mask]
df_good.head()

,Power,Detector
0,96,8
1,96,10
2,91,8
3,97,9
4,91,11


In [18]:
# df with only bad non anomaly values
df_bad = df[bad_mask]
df_bad.head()

,Power,Detector
44,94,6
45,93,7
46,95,6
47,94,7
48,97,5


In [19]:
# This is the feature vector that goes to the neural net

x_good = df_good.values
x_bad = df_bad.values

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
x_good_train, x_good_test = train_test_split(x_good, test_size = 0.25, random_state = 42)

In [22]:
print(len(x_good_train))
print(len(x_good_test))

790
264


In [23]:
# This is where model building starts
from sklearn import metrics

In [24]:
from tensorflow.keras.models import Sequential

In [25]:
from tensorflow.keras.layers import Dense

In [26]:
model = Sequential()
# use convolution instead of dense in case of images
model.add(Dense(10, input_dim = x_good.shape[1], activation = 'relu'))
model.add(Dense(3, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(x_good.shape[1]))
model.compile(loss = 'mean_squared_error', optimizer = 'adam')
model.summary()

model.fit(x_good_train, x_good_train, verbose = 1, epochs = 100)

pred = model.predict(x_good_test)
score1 = np.sqrt(metrics.mean_squared_error(pred, x_good_test))

pred = model.predict(x_good)
score2 = np.sqrt(metrics.mean_squared_error(pred, x_good))

pred = model.predict(x_bad)
score3 = np.sqrt(metrics.mean_squared_error(pred, x_bad))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                30        
                                                                 
 dense_1 (Dense)             (None, 3)                 33        
                                                                 
 dense_2 (Dense)             (None, 10)                40        
                                                                 
 dense_3 (Dense)             (None, 2)                 22        
                                                                 
Total params: 125
Trainable params: 125
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
25/25 [==============================] - 1s 3ms/step - loss: 4319.2998
Epoch 2/100
25/25 [==============================] - 0s 4ms/step - loss: 3980.5095
Epoch 3/100
25/25 [=====

In [31]:
print('In sample score (RMSE) :', score1)
print('Total data score (RMSE):', score2)
print('Bad data score (RMSE):', score3)

In sample score (RMSE) : 1.036731493319919
Total data score (RMSE): 1.034861767977408
Bad data score (RMSE): 2.5962585585742928


Conclusion : The error is way higher for the bad training data